## Примерная оценка размера БД

Расчет включает только объем данных без учета индексов, выравнивания и прочих накладных расходов

### Размер полей

#### Таблица `grid`
```sql
SELECT 
(round(avg(pg_column_size(g.id)))) AS id,
(round(avg(st_memsize(g.geometry)))) AS geometry
FROM grid g 
```

#### Таблица `records`
```sql
SELECT 
round(avg(pg_column_size(r.temperature))) AS temperature,
round(avg(pg_column_size(r.pressure))) AS pressure,
round(avg(pg_column_size(r.u_wind))) AS u_wind,
round(avg(pg_column_size(r.v_wind))) AS v_wind,
round(avg(pg_column_size(r.r_humidity))) AS r_humidity,
round(avg(pg_column_size(r.c_rain))) AS c_rain,
round(avg(pg_column_size(r.visibility))) AS visibility,
round(avg(pg_column_size(r.is_ground))) AS is_ground,
round(avg(pg_column_size(r.date_time))) AS date_time
FROM records r 
```

### Количество записей

Для таблицы `grid` количество записей `grid_row_count` зависит от разрешения `grid_resolution` и неизменно после инициализации

Для таблицы `record` количество записей `record_row_count` равно `66` * `grid_row_count` - модель GFS делает серию прогнозов на 180 часов с шагом в 3 часа с обновлением раз в 6 часов (0,6,12,18),
соответственно имеем `60` (180/3) прогнозов плюс `6` дополнительных к концу дня (еще по два прогноза для трех циклов: 6,12 и 18 часов) умноженное на число ячеек модели

In [63]:
def size_pretty(size:float)->str:
    prefix = ["","K","M","G","T"]
    res = size
    pID = 0
    while res >= 1024:
        res /= 1024
        pID += 1
        
    return f"{round(res,2)} {prefix[pID]}Byte"

In [64]:
grid_fields_size = {
	"id" : 8,
	"geometry" : 120
}

record_fields_sizes = {
	"id" : 8,
	"temperature" : 13,
	"grid_id" : 8,
	"pressure" : 11,
	"u_wind" : 13,
	"v_wind" : 13,
	"r_humidity" : 11,
	"c_rain" : 3,
	"visibility" : 12,
	"is_ground" : 1,
	"date_time" : 8
}

grid_resolution = 0.5 #[1.0,0.5,0.25][1]
forecast_resolution = 3 #hour
forecast_interval = 180 + 18  #hour


grid_row_count = (180 / grid_resolution + 1) * (360.0 / grid_resolution) # 360 градусов долгот, +90/-90 широт плюс экватор
record_row_count = (forecast_interval / forecast_resolution) * grid_row_count

grid_row_size = sum([s for _,s in grid_fields_size.items()])
records_row_size = sum([s for _,s in record_fields_sizes.items()])

grid_table_size = grid_row_size * grid_row_count
record_table_size = records_row_size * record_row_count

total_size = record_table_size + grid_table_size


print(f"Grid table:	{size_pretty(grid_table_size)}")
print(f"Record table:	{size_pretty(record_table_size)}")
print("-" * 10)
print(f"Total:		{size_pretty(total_size)}")

Grid table:	31.73 MByte
Record table:	1.61 GByte
----------
Total:		1.64 GByte
